In [1]:
from langchain.agents import AgentExecutor, create_tool_calling_agent
from langchain_core.runnables import RunnablePassthrough,RunnableLambda, Runnable, RunnableParallel,RunnableConfig
from langchain_core.messages import AIMessage
from dotenv import load_dotenv,find_dotenv
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_openai import ChatOpenAI
from langchain.tools import tool
from langchain_core.prompts import ChatPromptTemplate,SystemMessagePromptTemplate, HumanMessagePromptTemplate
from langchain_core.output_parsers import JsonOutputParser
from operator import itemgetter
import json

In [2]:
load_dotenv(find_dotenv("../.env"))

True

In [10]:
llmGemini=ChatGoogleGenerativeAI(model="gemini-2.0-flash-001")
llmOpenAI=ChatOpenAI(model="gpt-4o")

In [4]:
@tool
def complexTool(intArg: int, floatArg: float, dictArg: dict) -> int:
    """
        Do something complex with a complex tool
    """
    return intArg*floatArg

In [11]:
llmWithTools=llmOpenAI.bind_tools(tools=[complexTool])

In [6]:
question="Use Complex Tool. The args are 5, 2.1 and an empty dictionary. Don't forget dict_arg"

In [12]:
response=llmWithTools.invoke(input=question)

In [13]:
response.tool_calls

[{'name': 'complexTool',
  'args': {'intArg': 5, 'floatArg': 2.1},
  'id': 'call_RvuNdCY6MHuMpTkL4tdSaxeL',
  'type': 'tool_call'}]

In [14]:
chain=llmWithTools|RunnableLambda(lambda msg: msg.tool_calls[0]['args'])|complexTool

In [15]:
chain.invoke(input=question)

ValidationError: 1 validation error for complexTool
dictArg
  Field required [type=missing, input_value={'intArg': 5, 'floatArg': 2.1}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.11/v/missing

<h3> Try Except Tool Call</h3>

In [16]:
def tryExceptToolCall(toolArgs: dict, config=RunnableConfig) -> Runnable:
    try:
        return complexTool.invoke(
            input=toolArgs,
            config=RunnableConfig
        )
    except Exception as e:
        return f"{toolArgs}:{type(e)}:{e}"
        return f"Calling Tool with Arguments \n\n{toolArgs}\n\nraised the following error\n\n{type(e):e}"

In [17]:
chain=llmWithTools|RunnableLambda(lambda msg: msg.tool_calls[0]['args'])|tryExceptToolCall 

In [18]:
chain.invoke(input=question)

"{'intArg': 5, 'floatArg': 2.1}:<class 'TypeError'>:unbound method dict.items() needs an argument"

<h3> Falling to a Better Model (Fallbacks) </h3>

In [20]:
llm=ChatGoogleGenerativeAI(model="gemini-1.5-flash")
betterllm=ChatOpenAI(model="gpt-3.5-turbo")

In [21]:
llmWithTools=llm.bind_tools(tools=[complexTool])
betterllmWithTools=betterllm.bind_tools(tools=[complexTool])

In [22]:
chain = llmWithTools | (lambda msg: msg.tool_calls[0]["args"]) | complexTool
betterChain=betterllmWithTools | (lambda msg: msg.tool_calls[0]["args"]) | complexTool

In [38]:
chainWithFallback=chain.with_fallbacks(fallbacks=[betterChain])

In [39]:
chainWithFallback.invoke(input=question)

10.5